In [1]:
import pandas as pd
import os
from pathlib import Path
from sqlalchemy import create_engine

In [31]:
db_merged = 'wallapop_merged.db'

def delete_if_db_exists():
    if os.path.isfile(f'{db_merged}'):
        os.remove(f'{db_merged}')

def create_db_list():
    dbs_files = sorted(Path().glob('*.db'))
    dbs_list = [db.name for db in dbs_files]
    return dbs_list


def add_databases_to_dataframe(dbs_list):
    df_list = []
    for i, db in enumerate(dbs_list):
        if db != db_merged:
            current_engine = create_engine(f'sqlite:///{db}')
            current_df = pd.read_sql('cars', current_engine)
            df_list.append(current_df)
    
    merged_df = pd.concat(df_list)
    merged_df = merged_df.drop_duplicates()
    print(f'{db_merged} -> {len(merged_df)}')
    return merged_df


def export_db(merged_df):
    engine = create_engine(f'sqlite:///{db_merged}')
    merged_df.to_sql('cars', engine, index=False)


def main():
    delete_if_db_exists()
    dbs_list = create_db_list()
    merged_df = add_databases_to_dataframe(dbs_list)
    export_db(merged_df)
    

main()

wallapop_merged.db -> 912
